In [79]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [2]:
seed_list_categories = pd.read_excel('seedlist_urls.xlsx')
seed_list_products = pd.read_excel('scraped_product_urls.xlsx')
df = seed_list_products.merge(seed_list_categories, on='url', how='left')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415 entries, 0 to 414
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_name  415 non-null    object
 1   item_url   415 non-null    object
 2   price      414 non-null    object
 3   url        415 non-null    object
 4   category   415 non-null    object
dtypes: object(5)
memory usage: 19.5+ KB


In [3]:
df

,item_name,item_url,price,url,category
0,Kirkland Signature Peanut Butter Filled Pretze...,https://www.costco.com/kirkland-signature-pean...,$11.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels
1,"Frito Lay Classic Mix, 1 oz, Variety Pack, 54-...",https://www.costco.com/frito-lay-classic-mix%2...,$23.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels
2,"Snyder's of Hanover Mini Pretzels, 0.9 oz, 60-...",https://www.costco.com/snyder's-of-hanover-min...,$13.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels
3,"Frito Lay Oven Baked Mix, Variety Pack, 30-count",https://www.costco.com/frito-lay-oven-baked-mi...,$21.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels
4,"Popchips Potato Chips, Variety Pack, 0.8 oz, 3...",https://www.costco.com/popchips-potato-chips%2...,$16.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels
...,...,...,...,...,...
410,"Life Savers Gummy Candy, Collisions, Share Siz...",https://www.costco.com/life-savers-gummy-candy...,$29.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy
411,"E.Frutti Gummi Apple, 3.5 oz, 12-count",https://www.costco.com/e.frutti-gummi-apple%2c...,$13.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy
412,"Jolly Rancher Original Gummies, 7 oz, 12-count",https://www.costco.com/jolly-rancher-original-...,$26.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy
413,"Mamba Fruit Chews, 2.8 oz, 24-count",https://www.costco.com/mamba-fruit-chews%2c-2....,$19.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy


# Develop the scraper

## Experiments

In [6]:
# Create a directory to save
os.makedirs('costco_images', exist_ok=True)

In [70]:
headers = {
}

In [103]:
options = Options()
options.headless = True  # Run in headless mode

# Initialize the WebDriver
service = Service('/Users/anna.zemit/Downloads/chromedriver-mac-arm64/chromedriver')  
driver = webdriver.Chrome(service=service, options=options)

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  


In [107]:
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": headers['User-Agent']})
driver.execute_cdp_cmd('Network.enable', {})
driver.execute_cdp_cmd('Network.setExtraHTTPHeaders', {'headers': headers})


{}

In [108]:
driver.get(product_url)
time.sleep(5)

In [109]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [111]:
# Find the main product image URL
img_element = driver.find_element(By.ID, 'RICHFXViewerContainer___richfx_id_0_zoomImage')
img_url = img_element.get_attribute('src')

In [112]:
img_url

'https://richmedia.ca-richimage.com/ImageDelivery/imageService?profileId=12026540&id=1152515&recipeId=728'

In [113]:
driver.quit()

## Scraper

In [114]:
def save_image(img_url, file_path):
    try:
        img_response = requests.get(img_url, headers=headers, stream=True)
        img_response.raise_for_status()
        
        with open(file_path, 'wb') as file:
            for chunk in img_response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Image saved to {file_path}")
    except Exception as e:
        print(f"Error saving image from {img_url}: {e}")

In [116]:
def get_image_url(product_url):
    try:
        # Set up Chrome options
        options = Options()
        options.headless = True  # Run in headless mode

        # Initialize the WebDriver
        service = Service('/Users/anna.zemit/Downloads/chromedriver-mac-arm64/chromedriver')  
        driver = webdriver.Chrome(service=service, options=options)

        # Use CDP to add headers
        driver.execute_cdp_cmd('Network.setUserAgentOverride', {
            "userAgent": headers['User-Agent']
        })
        driver.execute_cdp_cmd('Network.enable', {})
        driver.execute_cdp_cmd('Network.setExtraHTTPHeaders', {'headers': headers})

        # Open the product URL
        driver.get(product_url)
        time.sleep(5)  # Wait for the page to fully load

        # Find the main product image URL
        img_element = driver.find_element(By.ID, 'RICHFXViewerContainer___richfx_id_0_zoomImage')
        img_url = img_element.get_attribute('src')

        driver.quit()
        return img_url

    except Exception as e:
        print(f"Error fetching {product_url}: {e}")
        return None

In [118]:
for _, row in df[1:].iterrows():
    img_url = get_image_url(row['item_url'])
    if img_url:
        img_name = row['item_name'] + '.jpg'
        save_path = os.path.join('costco_images', img_name)
        save_image(img_url, save_path)

/Users/anna.zemit/opt/anaconda3/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  """


Image saved to costco_images/Frito Lay Classic Mix, 1 oz, Variety Pack, 54-count.jpg
Image saved to costco_images/Snyder's of Hanover Mini Pretzels, 0.9 oz, 60-count.jpg
Image saved to costco_images/Frito Lay Oven Baked Mix, Variety Pack, 30-count.jpg
Image saved to costco_images/Popchips Potato Chips, Variety Pack, 0.8 oz, 30-count.jpg
Image saved to costco_images/Quaker Rice Crisps, Variety Pack, 36-count.jpg
Image saved to costco_images/Pringles Grab & Go Potato Crisps, Variety, 36-count.jpg
Image saved to costco_images/Frito Lay Select Mix, Variety Pack, 30-count.jpg
Image saved to costco_images/Doritos Tortilla Chips, Nacho Cheese, 1 oz, 50-count.jpg
Image saved to costco_images/Frito Lay Classic Mix, Variety Pack, 30-count.jpg
Image saved to costco_images/Sonoma Creamery Pepper Jack Crisps, 10 oz.jpg
Image saved to costco_images/Chex Mix, Traditional, 1.75 oz, 42-count.jpg
Image saved to costco_images/Cheez-It White Cheddar Baked Snack Cracker, 1.5 oz, 45-count.jpg
Image saved to

Image saved to costco_images/Loacker Classic Crispy Wafers, Hazelnut, 1.59 oz, 12 Count.jpg
Image saved to costco_images/Nilla Wafers Cookies, 15 oz, 2 Count.jpg
Image saved to costco_images/St Michel Madeleine, Classic French Sponge Cake 100 - count.jpg
Image saved to costco_images/Lenny & Larry's The Complete Cookie, White Chocolate Flavored Macadamia, 4 oz, 12 count.jpg
Image saved to costco_images/Mary Macleod's Gluten Free Shortbread Cookies Mixed Assortment 8-Pack.jpg
Image saved to costco_images/Mary Macleod's Assorted Shortbread Cookies, 2-pack, 65 cookies per jar.jpg
Image saved to costco_images/Kirkland Signature Organic Roasted Seaweed, 0.6 oz, 10-count.jpg
Image saved to costco_images/Kirkland Signature Organic Animal Crackers, 4 lbs.jpg
Image saved to costco_images/Pepperidge Farm Goldfish Crackers, Cheddar, 1 oz, 45-count.jpg
Image saved to costco_images/Crunchmaster Multi-Grain Crackers, 28 oz.jpg
Image saved to costco_images/Pepperidge Farm Goldfish Snacks, Variety Pack

Image saved to costco_images/Country Archer Grass-Fed Beef Stick Minis, Original, 0.5 oz, 28-Count.jpg
Image saved to costco_images/Jack Link's Big Beef Jerky Stick, Original, 0.92 oz, 20-count.jpg
Image saved to costco_images/Golden Island Pork Snack Bites, Korean Barbecue, 1.5 oz, 12-count.jpg
Image saved to costco_images/Jack Link's All American Beef Stick, Beef & Cheese, 1.2 oz, 16-count.jpg
Image saved to costco_images/Old Trapper Beef Jerky, Peppered, 10 oz.jpg
Image saved to costco_images/Think Jerky Grass Fed Beef Sticks, Original, 1 oz, 20-count.jpg
Image saved to costco_images/Slim Jim Smoked Snack Stick, Original, 0.28 oz, 120-count.jpg
Image saved to costco_images/Old Trapper Beef Jerky, Old Fashioned, 10 oz.jpg
Image saved to costco_images/Golden Island Korean Barbecue Pork Jerky, 14.5 oz.jpg
Image saved to costco_images/Pacific Gold Beef Jerky, Variety Pack, 15 oz, 12-count.jpg
Image saved to costco_images/Pacific Gold Original Beef Jerky, 1.25 oz, 12-count.jpg
Image save

Image saved to costco_images/The Worthy Crumb, Mini Muffins, Blueberry, 2.6 oz, 16-Count.jpg
Image saved to costco_images/Bon Appetit, Banana Bread Slice, 4 oz, 8-Count.jpg
Image saved to costco_images/Bon Appetit, Gourmet Danish, Variety Pack, 3 oz, 18-Count.jpg
Image saved to costco_images/The Worthy Crumb, Mini Muffins, Chocolate Chip, 2.6 oz, 16 Count.jpg
Image saved to costco_images/Hostess Cinnamon Streusel Coffee Cakes, 1.45 oz, 32-Count.jpg
Image saved to costco_images/JJ's Bakery, Lightly Glazed Apple Pie, 4 oz, 16-Count.jpg
Image saved to costco_images/Hostess Donettes, Variety Pack, 1.52 oz, 32-count.jpg
Image saved to costco_images/El Camino Real Bakery, Organic Coconut Bites, 1 lb 9 oz.jpg
Image saved to costco_images/Bon Appetit, Cheese Coffee Bar Cake, 4 oz, 8-Count.jpg
Image saved to costco_images/Marinela, Gansito Snack Cakes, 1.76 oz, 24-Count.jpg
Error fetching https://www.costco.com/bon-appetit%2c-mini-chocolate-donuts%2c-6-pack%2c-8-count.product.100856769.html: Me

Image saved to costco_images/G2G 3-pack Peanut Butter Coconut Chocolate Protein Bars 24-count.jpg
Image saved to costco_images/No Sugar Keto Bar (12 count) 2-pack Chocolate Peanut Butter.jpg
Image saved to costco_images/KIND Bar, Peanut Butter Dark Chocolate, 1.4 oz, 15-count.jpg
Image saved to costco_images/G2G 3-pack Almond Chocolate Chip Protein Bars, 24-count.jpg
Image saved to costco_images/The Gluten Free Brothers Variety Bites, 6 count.jpg
Error saving image from https://richmedia.ca-richimage.com/ImageDelivery/imageService?profileId=12026540&id=1892423&recipeId=728: ('Connection aborted.', OSError(0, 'Error'))
Image saved to costco_images/No Sugar METABAR Chocolate Caramel and Peanut Flavor 12-count, 2-pack.jpg
Image saved to costco_images/No Sugar Shellz Dark Chocolatey Caramel Pecan 25-count, 2-pack.jpg
Image saved to costco_images/No Sugar Keto Bar Chocolate Chip Cookie Dough  (12-count) 2-Pack.jpg
Image saved to costco_images/No Sugar Keto Bar (12-count) 2-pack Cookies & Cr

Image saved to costco_images/Milky Way Full Size Chocolate Candy, Caramel, 1.84 oz, 36-count.jpg
Image saved to costco_images/Rocky Mountain Chocolate Dark Chocolate, 2 lbs..jpg
Image saved to costco_images/Butterfinger Candy Bar, 1.9 oz, 36-count.jpg
Image saved to costco_images/Reese's Sticks, King Size, 3 oz, 24-count.jpg
Image saved to costco_images/Reese's Fast Break, King Size, 3.5 oz 18-count.jpg
Image saved to costco_images/Wild West Assorted Chocolate Box, 24.26 oz. total.jpg
Image saved to costco_images/Rocky Mountain Chocolate Party Pack Assortment, 1.5 lb Assortment.jpg
Image saved to costco_images/Snickers Chocolate Candy Bars, Share Size, 3.29 oz, 24 count.jpg
Image saved to costco_images/3 Musketeers Chocolate Candy Bar, Full Size, 1.92 oz, 36-count.jpg
Image saved to costco_images/Bouchard Belgian Probiotic Chocolate 1.1 lb 2-pack.jpg
Image saved to costco_images/Rocky Mountain Chocolate Deluxe Tower.jpg
Image saved to costco_images/Hershey's, Cookies N' Creme, 1.55 oz,

Image saved to costco_images/Mentos Pure Fresh Sugar Free Gum, Fresh Mint, 15 Pieces, 10-count.jpg
Image saved to costco_images/Ice Breakers Ice Cubes Sugar Free Gum, Spearmint, 40 pieces, 4-count.jpg
Image saved to costco_images/Ice Breakers Ice Cubes Sugar Free Gum, Wintergreen, 40 pieces, 4-count.jpg
Image saved to costco_images/Trident Sugar Free Gum, Wintergreen, 14 Sticks, 12-count.jpg
Error fetching https://www.costco.com/ice-breakers-ice-cubes-sugar-free-gum%2c-arctic-grape%2c-40-pieces%2c-4-ct%2c-160-pieces.product.100946299.html: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="RICHFXViewerContainer___richfx_id_0_zoomImage"]"}
  (Session info: chrome-headless-shell=125.0.6422.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104d3a4c8 chromedriver + 4302024
1   chromedrive

In [ ]:
# https://www.costco.com/tim's-cascade-potato-chips%2c-1.5-oz%2c-variety-pack%2c-30-count.product.100447346.html:
# https://www.costco.com/meiji-hello-panda-chocolate-creme-filled-cookies%2c-0.75-oz%2c-32-count.product.100368214.html:
# https://www.costco.com/annie's-organic-bunny-snack-pack-baked-crackers-and-graham-snacks%2c-1.07-oz%2c-variety-pack%2c-36-count.product.4000045056.html:
# https://www.costco.com/gogo-squeez%2c-organic-applesauce%2c-variety-pack%2c-3.2-oz%2c-28-count.product.4000203095.html:
# https://www.costco.com/kirkland-signature-premium-extra-thick-steak-strips%2c-12-oz.product.100333696.html:
# https://www.costco.com/kirkland-signature-organic-pine-nuts%2c-1.5-lbs.product.100334735.html:
# https://www.costco.com/bon-appetit%2c-mini-chocolate-donuts%2c-6-pack%2c-8-count.product.100856769.html:
# https://www.costco.com/g2g-protein-bar-variety-3-pack-24-total-bars.product.4000105597.html:
# https://www.costco.com/york-dark-chocolate-peppermint-patties%2c-84-oz%2c-175-count.product.100381541.html:
# https://www.costco.com/chocolate-moonshine-co.-belgian-artisan-black-raspberry-bark%2c-20-oz..product.4000232122.html:
# https://www.costco.com/trident-sugar-free-gum%2c-mint-bliss%2c-14-pieces%2c-15-count.product.100438351.html:
# https://www.costco.com/ice-breakers-ice-cubes-sugar-free-gum%2c-arctic-grape%2c-40-pieces%2c-4-ct%2c-160-pieces.product.100946299.html:


In [119]:
df['item_name'] = df['item_name'].apply(lambda x: x.replace('/', ' '))

In [127]:
# Define the path to the images folder
images_folder = 'costco_images'

def image_exists(item_name):
    image_path = os.path.join(images_folder, f"{item_name}.jpg")
    return os.path.exists(image_path)

# Apply the function to the DataFrame to create the 'image' column
df['image'] = df['item_name'].apply(image_exists)


In [128]:
df[df.image == False]

,item_name,item_url,price,url,category,image
26,"Tim's Cascade Potato Chips, 1.5 oz, Variety Pa...",https://www.costco.com/tim's-cascade-potato-ch...,$15.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels,False
65,Meiji Hello Panda Chocolate Creme Filled Cooki...,https://www.costco.com/meiji-hello-panda-choco...,$11.99,https://www.costco.com/cookies.html,Cookies,False
79,Annie's Organic Bunny Snack Pack Baked Cracker...,https://www.costco.com/annie's-organic-bunny-s...,$16.99,https://www.costco.com/crackers.html,Crackers,False
109,"GoGo SqueeZ, Organic Applesauce, Variety Pack,...",https://www.costco.com/gogo-squeez%2c-organic-...,$14.99,https://www.costco.com/fruit-snacks-applesauce...,Fruit Snacks & Applesauce,False
115,Kirkland Signature Premium Extra Thick Steak S...,https://www.costco.com/kirkland-signature-prem...,$12.99,https://www.costco.com/jerky.html,Jerky & Dried Meats,False
152,"Kirkland Signature Organic Pine Nuts, 1.5 lbs",https://www.costco.com/kirkland-signature-orga...,$29.99,https://www.costco.com/nuts.html,Nuts & Seeds,False
203,"Bon Appetit, Mini Chocolate Donuts, 6-Pack, 8-...",https://www.costco.com/bon-appetit%2c-mini-cho...,$13.99,https://www.costco.com/pastries.html,Pastries & Muffins,False
261,G2G Protein Bar Variety 3-pack (24 Total Bars),https://www.costco.com/g2g-protein-bar-variety...,$49.99,https://www.costco.com/snacks-bars.html,"Protein, Breakfast & Snack Bars",False
300,"York Dark Chocolate Peppermint Patties, 84 oz,...",https://www.costco.com/york-dark-chocolate-pep...,$19.99,https://www.costco.com/chocolates.html,Chocolates,False
354,Chocolate Moonshine Co. Belgian Artisan Black ...,https://www.costco.com/chocolate-moonshine-co....,$44.99,https://www.costco.com/chocolates.html,Chocolates,False


In [129]:
scraped = pd.read_csv('scraped_star_rating.csv')

In [130]:
# Initialize a new DataFrame with the desired structure
result_df = pd.DataFrame()

# Add unique item URLs
result_df['item_url'] = df['item_url'].unique()

# Initialize columns for each star rating with 0
result_df['one_star'] = 0
result_df['two_stars'] = 0
result_df['three_stars'] = 0
result_df['four_stars'] = 0
result_df['five_stars'] = 0

In [141]:
def count_reviews(url, stars, reviews):
    counts = {'one_star': 0, 'two_stars': 0, 'three_stars': 0, 'four_stars': 0, 'five_stars': 0}
    for star, review in zip(stars, reviews):
        if star == ' 1★ stars ':
            counts['one_star'] += review
        elif star == ' 2★ stars ':
            counts['two_stars'] += review
        elif star == ' 3★ stars ':
            counts['three_stars'] += review
        elif star == ' 4★ stars ':
            counts['four_stars'] += review
        elif star == ' 5★ stars ':
            counts['five_stars'] += review
    return counts

In [143]:
for index, row in result_df.iterrows():
    url = row['item_url']
    filtered_df = scraped[scraped['item_url'] == url]
    stars = filtered_df['stars'].tolist()
    reviews = filtered_df['reviews'].tolist()
    counts = count_reviews(url, stars, reviews)
    result_df.at[index, 'one_star'] = counts['one_star']
    result_df.at[index, 'two_stars'] = counts['two_stars']
    result_df.at[index, 'three_stars'] = counts['three_stars']
    result_df.at[index, 'four_stars'] = counts['four_stars']
    result_df.at[index, 'five_stars'] = counts['five_stars']


In [144]:
result_df

,item_url,one_star,two_stars,three_stars,four_stars,five_stars
0,https://www.costco.com/kirkland-signature-pean...,78,44,120,410,3588
1,https://www.costco.com/frito-lay-classic-mix%2...,0,0,0,0,0
2,https://www.costco.com/snyder's-of-hanover-min...,17,14,24,72,641
3,https://www.costco.com/frito-lay-oven-baked-mi...,17,13,19,59,258
4,https://www.costco.com/popchips-potato-chips%2...,13,13,26,57,314
...,...,...,...,...,...,...
406,https://www.costco.com/life-savers-gummy-candy...,0,0,0,0,2
407,https://www.costco.com/e.frutti-gummi-apple%2c...,0,0,0,2,2
408,https://www.costco.com/jolly-rancher-original-...,0,0,0,0,0
409,https://www.costco.com/mamba-fruit-chews%2c-2....,0,0,0,0,0


In [150]:
df = df.merge(result_df, on='item_url', how='left')

In [153]:
def calculate_average_rating(row):
    total_reviews = row['one_star'] + row['two_stars'] + row['three_stars'] + row['four_stars'] + row['five_stars']
    if total_reviews == 0:
        return 0
    total_stars = (1 * row['one_star'] + 2 * row['two_stars'] + 3 * row['three_stars'] + 4 * row['four_stars'] + 5 * row['five_stars'])
    return total_stars / total_reviews

df['average_rating'] = df.apply(calculate_average_rating, axis=1)


In [160]:
df['reviews'] = df[['one_star', 'two_stars', 'three_stars', 'four_stars', 'five_stars']].sum(axis=1)


In [163]:
df['reviews'].describe()

count     415.000000
mean      366.115663
std       676.426345
min         0.000000
25%        18.000000
50%        86.000000
75%       403.000000
max      4847.000000
Name: reviews, dtype: float64

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415 entries, 0 to 414
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   item_name       415 non-null    object 
 1   item_url        415 non-null    object 
 2   price           414 non-null    object 
 3   url             415 non-null    object 
 4   category        415 non-null    object 
 5   image           415 non-null    bool   
 6   one_star        415 non-null    int64  
 7   two_stars       415 non-null    int64  
 8   three_stars     415 non-null    int64  
 9   four_stars      415 non-null    int64  
 10  five_stars      415 non-null    int64  
 11  average_rating  415 non-null    float64
 12  reviews         415 non-null    int64  
dtypes: bool(1), float64(1), int64(6), object(5)
memory usage: 42.6+ KB


In [162]:
df

,item_name,item_url,price,url,category,image,one_star,two_stars,three_stars,four_stars,five_stars,average_rating,reviews
0,Kirkland Signature Peanut Butter Filled Pretze...,https://www.costco.com/kirkland-signature-pean...,$11.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels,True,78,44,120,410,3588,4.741981,4240
1,"Frito Lay Classic Mix, 1 oz, Variety Pack, 54-...",https://www.costco.com/frito-lay-classic-mix%2...,$23.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels,True,0,0,0,0,0,0.000000,0
2,"Snyder's of Hanover Mini Pretzels, 0.9 oz, 60-...",https://www.costco.com/snyder's-of-hanover-min...,$13.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels,True,17,14,24,72,641,4.700521,768
3,"Frito Lay Oven Baked Mix, Variety Pack, 30-count",https://www.costco.com/frito-lay-oven-baked-mi...,$21.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels,True,17,13,19,59,258,4.442623,366
4,"Popchips Potato Chips, Variety Pack, 0.8 oz, 3...",https://www.costco.com/popchips-potato-chips%2...,$16.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels,True,13,13,26,57,314,4.527187,423
...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,"Life Savers Gummy Candy, Collisions, Share Siz...",https://www.costco.com/life-savers-gummy-candy...,$29.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy,True,0,0,0,0,2,5.000000,2
411,"E.Frutti Gummi Apple, 3.5 oz, 12-count",https://www.costco.com/e.frutti-gummi-apple%2c...,$13.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy,True,0,0,0,2,2,4.500000,4
412,"Jolly Rancher Original Gummies, 7 oz, 12-count",https://www.costco.com/jolly-rancher-original-...,$26.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy,True,0,0,0,0,0,0.000000,0
413,"Mamba Fruit Chews, 2.8 oz, 24-count",https://www.costco.com/mamba-fruit-chews%2c-2....,$19.99,https://www.costco.com/hard-gummy-candy.html,Hard & Gummy Candy,True,0,0,0,0,0,0.000000,0


# Finalise the data into tabs


## Top 50

Top 50: isolate the 50 highest rated products, with a column category

In [164]:
top_50 = df[df.reviews >=18].sort_values(by='average_rating', ascending=False).head(50)
top_50

,item_name,item_url,price,url,category,image,one_star,two_stars,three_stars,four_stars,five_stars,average_rating,reviews
265,"G2G 3-pack Almond Chocolate Chip Protein Bars,...",https://www.costco.com/g2g-3-pack-almond-choco...,$49.99,https://www.costco.com/snacks-bars.html,"Protein, Breakfast & Snack Bars",True,0,0,0,0,22,5.000000,22
325,"Hershey's Milk Chocolate with Almonds, King Si...",https://www.costco.com/hershey's-milk-chocolat...,$34.99,https://www.costco.com/chocolates.html,Chocolates,True,0,0,0,0,29,5.000000,29
335,"Reese's Fast Break, King Size, 3.5 oz 18-count",https://www.costco.com/reese's-fast-break%2c-k...,$28.99,https://www.costco.com/chocolates.html,Chocolates,True,0,0,0,0,19,5.000000,19
46,"Nabisco Cookies & Crackers, Variety Pack, 1 oz...",https://www.costco.com/nabisco-cookies-%2526-c...,$14.99,https://www.costco.com/cookies.html,Cookies,True,0,0,1,0,20,4.904762,21
339,"3 Musketeers Chocolate Candy Bar, Full Size, 1...",https://www.costco.com/3-musketeers-chocolate-...,$39.99,https://www.costco.com/chocolates.html,Chocolates,True,0,0,0,5,38,4.883721,43
42,"Lay's Potato Chips, Barbecue, 1.5 oz, 64-count",https://www.costco.com/lay's-potato-chips%2c-b...,$49.99,https://www.costco.com/chips-pretzels.html,Chips & Pretzels,True,0,0,0,3,22,4.880000,25
302,Utah Truffles Dark Chocolate Truffles With Sea...,https://www.costco.com/utah-truffles-dark-choc...,$36.99,https://www.costco.com/chocolates.html,Chocolates,True,3,5,5,24,425,4.867965,462
94,Nutella & GO! Hazelnut and Cocoa Spread With B...,https://www.costco.com/nutella-%2526-go!-hazel...,$19.99,https://www.costco.com/crackers.html,Crackers,True,3,2,5,26,363,4.864662,399
368,"Altoids Breath Mints, Hard Peppermint Candy, 1...",https://www.costco.com/altoids-breath-mints%2c...,$26.99,https://www.costco.com/gum-mints.html,Gum & Mints,True,1,3,5,23,302,4.862275,334
330,"Amedei 70% Dark Chocolate, After Dinner Hotel ...",https://www.costco.com/amedei-70%25-dark-choco...,$104.99,https://www.costco.com/chocolates.html,Chocolates,True,0,0,0,4,24,4.857143,28


# Create an Excel file

In [165]:
# Create an Excel writer object

writer = pd.ExcelWriter('The 50 Most Loved Costco Snacks, Based on X Million Customers.xlsx')

In [166]:
top_50.to_excel(writer, sheet_name='Top 50', index=False)
df.to_excel(writer, sheet_name='All Data', index=False)


In [167]:
# Save the Excel file
writer.save()